In [ ]:
import sys
import os

import numpy as nplk
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

sys.path.insert(0, '../../')
import ccal
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

In [ ]:
nfeatures = 8
nelement = 7
features = ccal.support.make_random_features(nfeatures, nelement)
ref = ccal.support.make_random_features(1, nelement)
ccal.analyze.rank_features_against_reference(features, ref, nsampling=2, nperm=2)